In [1]:
import pandas as pd
import datetime

In [2]:
oms_data = pd.read_excel('../data/raw/Data Assignment File.xlsx', sheet_name='OPD')
paytm_data = pd.read_excel('../data/raw/Data Assignment File.xlsx', sheet_name='PAYTM EDC')

In [3]:
# Data Cleaning
oms_data.head(10)
# The first few rows of the oms-data are empty so we need to 
# skip them and make a new file. with the clean data
oms_data = oms_data.dropna(how='all')
oms_data.reset_index(drop=True, inplace=True)
oms_data_header = oms_data.iloc[0]
oms_data = oms_data[1:]
oms_data.reset_index(drop=True, inplace=True)
oms_data.columns = oms_data_header
oms_data.head(5)

,MPI,Transaction #,Receipt #,Receipt Date,Receipt Time,Type,Patient Name,Amount,From Advance,From Package,...,CHQ#/Transaction No.,Bank Transfer,Auth,Bank/Vendor,Amount Due,Paid By,Comments,User,Payment Aggregator Transaction ID,Transaction No
0,1000000100541195,OPD/JNR/23-24/CS/78006,OPD/JNR/23-24/RC/24180,2023-12-31 00:00:00,1900-01-01 23:40:50,OP Orders,Vandana,600,0,0,...,NaN,0,NaN,NaN,0,NaN,NaN,Manjunatha A,43144,NaN
1,1000000100991335,OPD/JNR/23-24/CS/78004,OPD/JNR/23-24/CON/RC/53825,2023-12-31 00:00:00,1900-01-01 23:06:50,OP Consult,Sai Vedansh,850,0,0,...,NaN,0,NaN,NaN,0,NaN,NaN,Manjunatha A,43141,NaN
2,1000000101881322,OPD/JNR/23-24/CS/78003,OPD/JNR/23-24/RC/24179,2023-12-31 00:00:00,1900-01-01 21:02:50,OP Orders,Jiyaan,300,0,0,...,123,0,NaN,upi,0,NaN,NaN,Manjunatha A,NaN,NaN
3,1000000101936174,OPD/JNR/23-24/CS/78001,OPD/JNR/23-24/CON/RC/53824,2023-12-31 00:00:00,1900-01-01 19:42:50,OP Consult,Ayeza Nishwa Fathima,1000,0,0,...,NaN,0,NaN,NaN,0,NaN,NaN,Salman Khan,43136,NaN
4,1000000100755069,OPD/JNR/23-24/CS/77999,OPD/JNR/23-24/RC/24176,2023-12-31 00:00:00,1900-01-01 19:29:50,OP Orders,B/O Shruthi,540,0,0,...,123,0,NaN,upi,0,NaN,NaN,Salman Khan,NaN,NaN


# Data Exploration

## OMS DATA

Required Fields are : 
1. **MPI**
2. **Transaction #**
3. **Receipt #**
4. **Receipt Date**
5. **Receipt Time**
6. **Type**
7. **Patient Name**
8. **Amount**
9. **From Advance**
10. ~~From Package~~ (as all the values are just 0) 
11. **Amount after Advance/Package** (Mostly not needed as it is redundent information but just keeping for reference)
12. **Cash** (Has some missing values need to fill them with 0)
13. **Card** (Has some missing values need to fill them with 0)
14. **CHQ/Wallet** (Has some missing values need to fill them with 0)
15. ~~CHQ#/Transaction No.~~ (Not needed as it is not a unique identifier and is redundant information) 
16. ~~Bank Transfer~~( Not needed as there are no transaction made from that mode at all)
17. ~~Auth~~ (Not needed as the whole column is blank)
18. **Bank/Vendor** (Mostly not needed but need to check if there is any use for it in future)
19. **Amount Due**(Mostly not needed but will be useful to detect any discrepancy in the data)
20. ~~Paid By~~ (Not useful empty Column)
21. **Comments**(Can be useful to detect any anomaly like Zero bills are raised by the vendor to keep a record if any amounts are zeroed by some referral of the doctor/employee discount/ etc)
22. **User** (might be useful for reconciliation of bills)
23. **Payment Aggregator Id** (might be useful for reconciliation of bills)
24. ~~Transaction No~~ (Not useful)


In [4]:
oms_data.columns

Index(['MPI', 'Transaction #', 'Receipt #', 'Receipt Date', 'Receipt Time',
       'Type', 'Patient Name', 'Amount', 'From Advance', 'From Package',
       'Amount after Advance/Package', 'Cash', 'Card', 'CHQ/Wallet',
       'CHQ#/Transaction No.', 'Bank Transfer', 'Auth', 'Bank/Vendor',
       'Amount Due', 'Paid By', 'Comments', 'User',
       'Payment Aggregator Transaction ID', 'Transaction No'],
      dtype='object', name=0)

In [5]:
# Drop all the unneccesary columns in oms_data dataframe:
oms_columns_to_drop = ['From Package', 'CHQ#/Transaction No.', 'Bank Transfer', 'Auth', 'Paid By', 'Transaction No']
oms_data = oms_data.drop(columns=oms_columns_to_drop)
oms_data.head()


,MPI,Transaction #,Receipt #,Receipt Date,Receipt Time,Type,Patient Name,Amount,From Advance,Amount after Advance/Package,Cash,Card,CHQ/Wallet,Bank/Vendor,Amount Due,Comments,User,Payment Aggregator Transaction ID
0,1000000100541195,OPD/JNR/23-24/CS/78006,OPD/JNR/23-24/RC/24180,2023-12-31 00:00:00,1900-01-01 23:40:50,OP Orders,Vandana,600,0,600,0,600,0,NaN,0,NaN,Manjunatha A,43144
1,1000000100991335,OPD/JNR/23-24/CS/78004,OPD/JNR/23-24/CON/RC/53825,2023-12-31 00:00:00,1900-01-01 23:06:50,OP Consult,Sai Vedansh,850,0,850,0,850,0,NaN,0,NaN,Manjunatha A,43141
2,1000000101881322,OPD/JNR/23-24/CS/78003,OPD/JNR/23-24/RC/24179,2023-12-31 00:00:00,1900-01-01 21:02:50,OP Orders,Jiyaan,300,0,300,0,0,300,upi,0,NaN,Manjunatha A,NaN
3,1000000101936174,OPD/JNR/23-24/CS/78001,OPD/JNR/23-24/CON/RC/53824,2023-12-31 00:00:00,1900-01-01 19:42:50,OP Consult,Ayeza Nishwa Fathima,1000,0,1000,0,1000,0,NaN,0,NaN,Salman Khan,43136
4,1000000100755069,OPD/JNR/23-24/CS/77999,OPD/JNR/23-24/RC/24176,2023-12-31 00:00:00,1900-01-01 19:29:50,OP Orders,B/O Shruthi,540,0,540,0,0,540,upi,0,NaN,Salman Khan,NaN


In [6]:
oms_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6680 entries, 0 to 6679
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   MPI                                6680 non-null   object
 1   Transaction #                      6680 non-null   object
 2   Receipt #                          6680 non-null   object
 3   Receipt Date                       6680 non-null   object
 4   Receipt Time                       6680 non-null   object
 5   Type                               6680 non-null   object
 6   Patient Name                       6680 non-null   object
 7   Amount                             6680 non-null   object
 8   From Advance                       6680 non-null   object
 9   Amount after Advance/Package       6467 non-null   object
 10  Cash                               6680 non-null   object
 11  Card                               6467 non-null   object
 12  CHQ/Wa

In [7]:
# fill all the empty cells with 0 in [Amount, From Advance, Amount after Advance/Package, Cash, Card, CHQ/Wallet, Amount Due] in the 

oms_columns_empty_cell_fill_with_zero = ['Amount', 'From Advance', 'Amount after Advance/Package', 'Cash', 'Card', 'CHQ/Wallet', 'Amount Due']

for column in oms_columns_empty_cell_fill_with_zero:
    oms_data[column] = oms_data[column].fillna(0)

oms_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6680 entries, 0 to 6679
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   MPI                                6680 non-null   object 
 1   Transaction #                      6680 non-null   object 
 2   Receipt #                          6680 non-null   object 
 3   Receipt Date                       6680 non-null   object 
 4   Receipt Time                       6680 non-null   object 
 5   Type                               6680 non-null   object 
 6   Patient Name                       6680 non-null   object 
 7   Amount                             6680 non-null   float64
 8   From Advance                       6680 non-null   float64
 9   Amount after Advance/Package       6680 non-null   float64
 10  Cash                               6680 non-null   int64  
 11  Card                               6680 non-null   float

In [8]:
paytm_data.head()

,Transaction_ID,Order_ID,Merchant_Unique_Ref,Merchant_Request_Id,Transaction_Date,Updated_Date,Transaction_Type,Status,MID,Merchant_Name,...,Gateway_Response_Code,Gateway_Response_Message,ARN_AVAILABILITY,IS_REVERSAL,Settle_Type,Merchant_Payment_Detail_1,Merchant_Subvention_Reversal_Amount,Cash_Advance_Emi_Reversal_Amount,Settlement_Bundle_Id,Settle_Factors
0,20231201010880000937176714792993815',2023120102461608140510820434',NaN,NaN,2023-12-01 02:45:40',2023-12-01 02:45:43',ACQUIRING',SUCCESS',CLOUDN45392268927328',NaN,...,NaN,NaN,NaN,NaN,INSTANT_BILL',NaN,NaN,NaN,NaN,NaN
1,20231201010910000937272200426767146',ORD1538522',NaN,NaN,2023-12-01 09:05:06',2023-12-01 09:05:08',ACQUIRING',SUCCESS',CLOUDN45392268927328',NaN,...,NaN,NaN,NaN,NaN,INSTANT_BILL',NaN,NaN,NaN,NaN,NaN
2,20231201010810000937237335752680236',ORD1538384',NaN,NaN,2023-12-01 06:46:33',2023-12-01 06:46:37',ACQUIRING',SUCCESS',CLOUDN45392268927328',NaN,...,NaN,NaN,NaN,NaN,INSTANT_BILL',NaN,NaN,NaN,NaN,NaN
3,20231201010930000937195176500533947',2023120103594308140610820434',NaN,NaN,2023-12-01 03:59:02',2023-12-01 03:59:05',ACQUIRING',SUCCESS',CLOUDN45392268927328',NaN,...,NaN,NaN,NaN,NaN,INSTANT_BILL',NaN,NaN,NaN,NaN,NaN
4,20231201010820000937194197378068590',ORD1538371',NaN,NaN,2023-12-01 03:55:08',2023-12-01 03:55:11',ACQUIRING',SUCCESS',CLOUDN45392268927328',NaN,...,NaN,NaN,NaN,NaN,INSTANT_BILL',NaN,NaN,NaN,NaN,NaN


In [9]:
# Need to drop all the columns which are NaN or empty in paytm_data dataframe.

for col in paytm_data.columns:
    if paytm_data[col].isnull().all() or paytm_data[col].empty:
        paytm_data.drop(columns=col,inplace=True)

paytm_data.head()

,Transaction_ID,Order_ID,Transaction_Date,Updated_Date,Transaction_Type,Status,MID,Amount,Channel,Payout_Date,...,Original_txn_value_before_promo,Auth_Code,RRN,Prepaid_Card,Bank/Gateway,Card_Scheme,User_Expected_Credit_Date,Pos_Date,Pos_Time,Settle_Type
0,20231201010880000937176714792993815',2023120102461608140510820434',2023-12-01 02:45:40',2023-12-01 02:45:43',ACQUIRING',SUCCESS',CLOUDN45392268927328',388.0,SYSTEM',2023-12-01 02:45:43',...,388.0,085174',000000081405',false',HFPP',MASTER',DEBIT_CARD:null',1201',024616',INSTANT_BILL'
1,20231201010910000937272200426767146',ORD1538522',2023-12-01 09:05:06',2023-12-01 09:05:08',ACQUIRING',SUCCESS',CLOUDN45392268927328',600.0,SYSTEM',2023-12-01 09:05:08',...,600.0,127986',000000041374',false',HFPP',VISA',CREDIT_CARD:null',1201',090508',INSTANT_BILL'
2,20231201010810000937237335752680236',ORD1538384',2023-12-01 06:46:33',2023-12-01 06:46:37',ACQUIRING',SUCCESS',CLOUDN45392268927328',150.0,SYSTEM',2023-12-01 06:46:37',...,150.0,029535',000000041371',false',HFPP',MASTER',CREDIT_CARD:null',1201',064636',INSTANT_BILL'
3,20231201010930000937195176500533947',2023120103594308140610820434',2023-12-01 03:59:02',2023-12-01 03:59:05',ACQUIRING',SUCCESS',CLOUDN45392268927328',226.0,SYSTEM',2023-12-01 03:59:05',...,226.0,026348',000000081406',false',HFPP',VISA',CREDIT_CARD:null',1201',035943',INSTANT_BILL'
4,20231201010820000937194197378068590',ORD1538371',2023-12-01 03:55:08',2023-12-01 03:55:11',ACQUIRING',SUCCESS',CLOUDN45392268927328',850.0,SYSTEM',2023-12-01 03:55:11',...,850.0,251481',000000041370',false',HFPP',VISA',CREDIT_CARD:null',1201',035457',INSTANT_BILL'


In [10]:
paytm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4652 entries, 0 to 4651
Data columns (total 33 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Transaction_ID                   4652 non-null   object 
 1   Order_ID                         4652 non-null   object 
 2   Transaction_Date                 4652 non-null   object 
 3   Updated_Date                     4652 non-null   object 
 4   Transaction_Type                 4652 non-null   object 
 5   Status                           4652 non-null   object 
 6   MID                              4652 non-null   object 
 7   Amount                           4652 non-null   float64
 8   Channel                          4652 non-null   object 
 9   Payout_Date                      4652 non-null   object 
 10  Settled_Date                     4652 non-null   object 
 11  Payment_Mode                     4652 non-null   object 
 12  Issuing_Bank        

In [11]:
paytm_data.columns

Index(['Transaction_ID', 'Order_ID', 'Transaction_Date', 'Updated_Date',
       'Transaction_Type', 'Status', 'MID', 'Amount', 'Channel', 'Payout_Date',
       'Settled_Date', 'Payment_Mode', 'Issuing_Bank', 'POS_ID',
       'External_Serial_No', 'Product_Code', 'Request_Type',
       'Link_Description', 'Credit/Debit_Card_Last_4_Digits',
       'Bank_Transaction_ID', 'Settled_Amount', 'Response_code',
       'Response_message', 'Original_txn_value_before_promo', 'Auth_Code',
       'RRN', 'Prepaid_Card', 'Bank/Gateway', 'Card_Scheme',
       'User_Expected_Credit_Date', 'Pos_Date', 'Pos_Time', 'Settle_Type'],
      dtype='object')

In [12]:

paytm_data_total_records = paytm_data['Bank_Transaction_ID'].size # 4652
paytm_data_unique_Bank_Transaction_ID_records = paytm_data['Bank_Transaction_ID'].unique().size #4532

print("Total Records : ", paytm_data_total_records , "\nUnique Bank Transaction IDs : ", paytm_data_unique_Bank_Transaction_ID_records)



Total Records :  4652 
Unique Bank Transaction IDs :  4532


> This means that there are around 120 duplicate records in the paytm_data dataframe with the same bank_transaction_id.

In [13]:
duplicate_bank_transaction_ids = set(paytm_data[paytm_data.duplicated(subset=['Bank_Transaction_ID'])]['Bank_Transaction_ID'])
for id in duplicate_bank_transaction_ids:
    print(id)

000000041693'
000000042288'
000000042946'
000000042564'
000000043126'
000000042496'
000000042472'
000000041773'
000000042260'
000000041737'
000000042303'
000000042070'
000000042258'
000000041761'
000000041785'
000000042677'
000000041926'
000000042981'
000000042600'
000000041742'
000000042597'
000000042960'
000000043117'
000000041422'
000000042306'
000000042194'
000000042285'
000000041767'
000000042153'
000000041736'
000000041821'
000000042291'
000000042261'
000000043056'
000000042097'
000000041429'
000000041539'
000000042939'
000000042997'
000000042784'
000000042729'
000000043052'
000000042172'
000000042407'
000000042433'
000000042167'
000000042209'
000000041488'
000000042913'
000000042578'
000000041766'
000000041765'
000000043144'
000000041677'
000000042384'
000000042259'
000000041681'
000000042622'
000000042452'
000000042029'
000000042666'
000000043018'
000000041764'
000000041913'
000000042085'
000000041822'
000000041997'
000000043143'
000000041654'
000000042143'
000000041928'
000000

In [14]:
paytm_data.columns

Index(['Transaction_ID', 'Order_ID', 'Transaction_Date', 'Updated_Date',
       'Transaction_Type', 'Status', 'MID', 'Amount', 'Channel', 'Payout_Date',
       'Settled_Date', 'Payment_Mode', 'Issuing_Bank', 'POS_ID',
       'External_Serial_No', 'Product_Code', 'Request_Type',
       'Link_Description', 'Credit/Debit_Card_Last_4_Digits',
       'Bank_Transaction_ID', 'Settled_Amount', 'Response_code',
       'Response_message', 'Original_txn_value_before_promo', 'Auth_Code',
       'RRN', 'Prepaid_Card', 'Bank/Gateway', 'Card_Scheme',
       'User_Expected_Credit_Date', 'Pos_Date', 'Pos_Time', 'Settle_Type'],
      dtype='object')

###  Data Cleaning for Paytm_EDC
- According to the analysis there are still some columns which are redundent like Amount and settled amount so one of the column can be dropped.
- Similarly upon inspection there are few other column like 
  -  Transaction_Type(all the entries are "ACQUIRING" so dropping the column)
  -  Status (only Contains "SUCCESS")
  -  MID (as the data is of only one merchant the merchantID is same for all the records therefore can be ignored)
  -  Amount (the data is same as Settled_Amount therefore we can drop one of the column)
  -  Channel (only contains "SYSTEM")
  -  Payout_Date (the data is same as Settled_Date therefore we can drop one of the column)
  -  Product_Code (only contains a single value)
  -  Request_Type (only contains "EDC")
  -  Link_Description (data is same as "POS_ID" so dropping one of the column)
  -  Response_code (only contains "01")
  -  Response_message (only contains "Txn Success")
  -  Original_txn_value_before_promo (the data is same as Settled_Amount therefore we can drop one of the column)
  -  RRN (has same information as that of "Bank_Transaction_ID" column so dropping one of them)
  -  Prepaid_Card (all the entries are "false" so dropping the column)
  -  Settle_Type (all the entries are "INSTANT_BILL" so dropping the column)

In [15]:
# Drop all the unneccesary columns in paytm_data dataframe:
paytm_columns_to_drop = ['Transaction_Type', 'Status', 'MID', 'Amount', 'Channel', 'Payout_Date', 'Product_Code', 'Request_Type', 'Link_Description', 'Response_code', 'Response_message', 'Original_txn_value_before_promo', 'RRN', 'Prepaid_Card', 'Settle_Type']
paytm_data = paytm_data.drop(columns=paytm_columns_to_drop)
paytm_data.head()

,Transaction_ID,Order_ID,Transaction_Date,Updated_Date,Settled_Date,Payment_Mode,Issuing_Bank,POS_ID,External_Serial_No,Credit/Debit_Card_Last_4_Digits,Bank_Transaction_ID,Settled_Amount,Auth_Code,Bank/Gateway,Card_Scheme,User_Expected_Credit_Date,Pos_Date,Pos_Time
0,20231201010880000937176714792993815',2023120102461608140510820434',2023-12-01 02:45:40',2023-12-01 02:45:43',2023-12-01 02:45:43',DEBIT_CARD',HDFC',10820434',3120110745724962821',6207',000000081405',388.0,085174',HFPP',MASTER',DEBIT_CARD:null',1201',024616'
1,20231201010910000937272200426767146',ORD1538522',2023-12-01 09:05:06',2023-12-01 09:05:08',2023-12-01 09:05:08',CREDIT_CARD',AXIS',10986499',3120112495789276165',4429',000000041374',600.0,127986',HFPP',VISA',CREDIT_CARD:null',1201',090508'
2,20231201010810000937237335752680236',ORD1538384',2023-12-01 06:46:33',2023-12-01 06:46:37',2023-12-01 06:46:37',CREDIT_CARD',HDFC',10986499',3120111849129772033',5667',000000041371',150.0,029535',HFPP',MASTER',CREDIT_CARD:null',1201',064636'
3,20231201010930000937195176500533947',2023120103594308140610820434',2023-12-01 03:59:02',2023-12-01 03:59:05',2023-12-01 03:59:05',CREDIT_CARD',ICICI',10820434',3120111078180630529',9002',000000081406',226.0,026348',HFPP',VISA',CREDIT_CARD:null',1201',035943'
4,20231201010820000937194197378068590',ORD1538371',2023-12-01 03:55:08',2023-12-01 03:55:11',2023-12-01 03:55:11',CREDIT_CARD',ICICI',10986499',3120111061469625345',9002',000000041370',850.0,251481',HFPP',VISA',CREDIT_CARD:null',1201',035457'


In [16]:
paytm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4652 entries, 0 to 4651
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Transaction_ID                   4652 non-null   object 
 1   Order_ID                         4652 non-null   object 
 2   Transaction_Date                 4652 non-null   object 
 3   Updated_Date                     4652 non-null   object 
 4   Settled_Date                     4652 non-null   object 
 5   Payment_Mode                     4652 non-null   object 
 6   Issuing_Bank                     4652 non-null   object 
 7   POS_ID                           4652 non-null   object 
 8   External_Serial_No               4652 non-null   object 
 9   Credit/Debit_Card_Last_4_Digits  4652 non-null   object 
 10  Bank_Transaction_ID              4652 non-null   object 
 11  Settled_Amount                   4652 non-null   float64
 12  Auth_Code           

In [17]:
oms_data.head()

,MPI,Transaction #,Receipt #,Receipt Date,Receipt Time,Type,Patient Name,Amount,From Advance,Amount after Advance/Package,Cash,Card,CHQ/Wallet,Bank/Vendor,Amount Due,Comments,User,Payment Aggregator Transaction ID
0,1000000100541195,OPD/JNR/23-24/CS/78006,OPD/JNR/23-24/RC/24180,2023-12-31 00:00:00,1900-01-01 23:40:50,OP Orders,Vandana,600.0,0.0,600.0,0,600.0,0.0,NaN,0,NaN,Manjunatha A,43144
1,1000000100991335,OPD/JNR/23-24/CS/78004,OPD/JNR/23-24/CON/RC/53825,2023-12-31 00:00:00,1900-01-01 23:06:50,OP Consult,Sai Vedansh,850.0,0.0,850.0,0,850.0,0.0,NaN,0,NaN,Manjunatha A,43141
2,1000000101881322,OPD/JNR/23-24/CS/78003,OPD/JNR/23-24/RC/24179,2023-12-31 00:00:00,1900-01-01 21:02:50,OP Orders,Jiyaan,300.0,0.0,300.0,0,0.0,300.0,upi,0,NaN,Manjunatha A,NaN
3,1000000101936174,OPD/JNR/23-24/CS/78001,OPD/JNR/23-24/CON/RC/53824,2023-12-31 00:00:00,1900-01-01 19:42:50,OP Consult,Ayeza Nishwa Fathima,1000.0,0.0,1000.0,0,1000.0,0.0,NaN,0,NaN,Salman Khan,43136
4,1000000100755069,OPD/JNR/23-24/CS/77999,OPD/JNR/23-24/RC/24176,2023-12-31 00:00:00,1900-01-01 19:29:50,OP Orders,B/O Shruthi,540.0,0.0,540.0,0,0.0,540.0,upi,0,NaN,Salman Khan,NaN


# Formatting the data in PAYTM_DATA 
- The data in the paytm_data dataframe are all having "'" at the end we are suppose to remove that.

In [18]:
# The entries in each cell are of type string and all of them are having ' in them. so its time to clean that

# This function removes all the ' in the string as it is the last character in the string.
def clean_string(string):
    return string[:-1]


In [19]:
columns_to_clean = list(paytm_data.columns)
columns_to_clean.remove("Settled_Amount")

for col in columns_to_clean:
    paytm_data[col] = paytm_data[col].apply(clean_string)

In [20]:
# Standardizing the date and time format in oms_data dataframe
def standardizeDatetime(date_obj,time_obj):
    # Create a new datetime object with the date from the date object and the time from the time object
    combined_datetime = datetime.datetime(date_obj.year, date_obj.month, date_obj.day, time_obj.hour, time_obj.minute, time_obj.second).strftime("%Y%m%d%H%M%S")
    return combined_datetime


# Standardizing the date and time format in paytm_data dataframe
def changeFormat(date):
    return datetime.datetime.strptime(date,"%Y-%m-%d %H:%M:%S").strftime("%Y%m%d%H%M%S")

def convertToUnixTimestamp(date):
    return datetime.datetime.strptime(date,"%Y%m%d%H%M%S").timestamp()

def fuzzyMatchString(unixTimestamp, cardAmount,chqWalletAmount=0):
    return str(int(unixTimestamp)) + f" {(cardAmount+chqWalletAmount):.2f}"

def finalAmountStr(amount1,amount2=0):
    return f"{(amount1+amount2):.2f}"

# Standardizing OMS_DATA

In [21]:
oms_data["Standardized DateTime"] = oms_data.apply(lambda x: standardizeDatetime(x["Receipt Date"], x["Receipt Time"]), axis=1)
oms_data["UnixTimestamp"] = oms_data["Standardized DateTime"].apply(convertToUnixTimestamp)
oms_data["UnixTimestampAmount"] = oms_data.apply(lambda x: fuzzyMatchString(x["UnixTimestamp"], x["Card"], x["CHQ/Wallet"]), axis=1)
oms_data["DateTimeAmount"] = oms_data.apply(lambda x: fuzzyMatchString(x["Standardized DateTime"], x["Card"], x["CHQ/Wallet"]), axis=1)
oms_data["FinalAmount"] = oms_data.apply(lambda x : finalAmountStr(x["Card"], x["CHQ/Wallet"]), axis=1)
oms_data.drop(columns=["Receipt Date", "Receipt Time"], inplace=True)

# Standardizing PAYTM_DATA

In [22]:
paytm_data["Standardized DateTime"] = paytm_data["Settled_Date"].apply(changeFormat)
paytm_data["UnixTimestamp"] = paytm_data["Standardized DateTime"].apply(convertToUnixTimestamp)
paytm_data["UnixTimestampAmount"] = paytm_data.apply(lambda x: fuzzyMatchString(x["UnixTimestamp"], x["Settled_Amount"]), axis=1)
paytm_data["DateTimeAmount"]= paytm_data.apply(lambda x : fuzzyMatchString(x['Standardized DateTime'], x["Settled_Amount"]), axis=1)
paytm_data["FinalAmount"] = paytm_data.apply(lambda x : finalAmountStr(x["Settled_Amount"]), axis=1)

paytm_data.drop(columns=["Transaction_Date","Updated_Date","Settled_Date","Pos_Date", "Pos_Time"], inplace=True)

In [23]:
oms_data.head(20)

,MPI,Transaction #,Receipt #,Type,Patient Name,Amount,From Advance,Amount after Advance/Package,Cash,Card,...,Bank/Vendor,Amount Due,Comments,User,Payment Aggregator Transaction ID,Standardized DateTime,UnixTimestamp,UnixTimestampAmount,DateTimeAmount,FinalAmount
0,1000000100541195,OPD/JNR/23-24/CS/78006,OPD/JNR/23-24/RC/24180,OP Orders,Vandana,600.0,0.0,600.0,0,600.0,...,NaN,0,NaN,Manjunatha A,43144,20231231234050,1.704046e+09,1704046250 600.00,20231231234050 600.00,600.00
1,1000000100991335,OPD/JNR/23-24/CS/78004,OPD/JNR/23-24/CON/RC/53825,OP Consult,Sai Vedansh,850.0,0.0,850.0,0,850.0,...,NaN,0,NaN,Manjunatha A,43141,20231231230650,1.704044e+09,1704044210 850.00,20231231230650 850.00,850.00
2,1000000101881322,OPD/JNR/23-24/CS/78003,OPD/JNR/23-24/RC/24179,OP Orders,Jiyaan,300.0,0.0,300.0,0,0.0,...,upi,0,NaN,Manjunatha A,NaN,20231231210250,1.704037e+09,1704036770 300.00,20231231210250 300.00,300.00
3,1000000101936174,OPD/JNR/23-24/CS/78001,OPD/JNR/23-24/CON/RC/53824,OP Consult,Ayeza Nishwa Fathima,1000.0,0.0,1000.0,0,1000.0,...,NaN,0,NaN,Salman Khan,43136,20231231194250,1.704032e+09,1704031970 1000.00,20231231194250 1000.00,1000.00
4,1000000100755069,OPD/JNR/23-24/CS/77999,OPD/JNR/23-24/RC/24176,OP Orders,B/O Shruthi,540.0,0.0,540.0,0,0.0,...,upi,0,NaN,Salman Khan,NaN,20231231192950,1.704031e+09,1704031190 540.00,20231231192950 540.00,540.00
5,1000000100755069,OPD/JNR/23-24/CS/77998,OPD/JNR/23-24/RC/24175,OP Orders,B/O Shruthi,2360.0,0.0,2360.0,0,0.0,...,upi,0,NaN,Salman Khan,NaN,20231231192350,1.704031e+09,1704030830 2360.00,20231231192350 2360.00,2360.00
6,1000000100755069,OPD/JNR/23-24/CS/77997,OPD/JNR/23-24/CON/RC/53823,OP Consult,B/O Shruthi,850.0,0.0,850.0,0,0.0,...,upi,0,NaN,Salman Khan,NaN,20231231191150,1.704030e+09,1704030110 850.00,20231231191150 850.00,850.00
7,1000000100367779,OPD/JNR/23-24/CS/77995,OPD/JNR/23-24/CON/RC/53821,OP Consult,Ankita Singhvi,850.0,0.0,850.0,0,850.0,...,NaN,0,NaN,Salman Khan,43126,20231231185250,1.704029e+09,1704028970 850.00,20231231185250 850.00,850.00
8,1000000100367779,OPD/JNR/23-24/CS/77994,OPD/JNR/23-24/RC/24174,OP Orders,Ankita Singhvi,1680.0,0.0,1680.0,0,1680.0,...,NaN,0,NaN,Salman Khan,43125,20231231185150,1.704029e+09,1704028910 1680.00,20231231185150 1680.00,1680.00
9,1000000100367779,OPD/JNR/23-24/CS/77993,OPD/JNR/23-24/RC/24173,OP Orders,Ankita Singhvi,2900.0,0.0,2900.0,0,2900.0,...,NaN,0,NaN,Salman Khan,43122,20231231184950,1.704029e+09,1704028790 2900.00,20231231184950 2900.00,2900.00


In [24]:
paytm_data.head()

,Transaction_ID,Order_ID,Payment_Mode,Issuing_Bank,POS_ID,External_Serial_No,Credit/Debit_Card_Last_4_Digits,Bank_Transaction_ID,Settled_Amount,Auth_Code,Bank/Gateway,Card_Scheme,User_Expected_Credit_Date,Standardized DateTime,UnixTimestamp,UnixTimestampAmount,DateTimeAmount,FinalAmount
0,20231201010880000937176714792993815,2023120102461608140510820434,DEBIT_CARD,HDFC,10820434,3120110745724962821,6207,000000081405,388.0,085174,HFPP,MASTER,DEBIT_CARD:null,20231201024543,1.701379e+09,1701378943 388.00,20231201024543 388.00,388.00
1,20231201010910000937272200426767146,ORD1538522,CREDIT_CARD,AXIS,10986499,3120112495789276165,4429,000000041374,600.0,127986,HFPP,VISA,CREDIT_CARD:null,20231201090508,1.701402e+09,1701401708 600.00,20231201090508 600.00,600.00
2,20231201010810000937237335752680236,ORD1538384,CREDIT_CARD,HDFC,10986499,3120111849129772033,5667,000000041371,150.0,029535,HFPP,MASTER,CREDIT_CARD:null,20231201064637,1.701393e+09,1701393397 150.00,20231201064637 150.00,150.00
3,20231201010930000937195176500533947,2023120103594308140610820434,CREDIT_CARD,ICICI,10820434,3120111078180630529,9002,000000081406,226.0,026348,HFPP,VISA,CREDIT_CARD:null,20231201035905,1.701383e+09,1701383345 226.00,20231201035905 226.00,226.00
4,20231201010820000937194197378068590,ORD1538371,CREDIT_CARD,ICICI,10986499,3120111061469625345,9002,000000041370,850.0,251481,HFPP,VISA,CREDIT_CARD:null,20231201035511,1.701383e+09,1701383111 850.00,20231201035511 850.00,850.00


In [25]:
paytm_data['Bank_Transaction_ID'] = paytm_data['Bank_Transaction_ID'].apply(lambda x : int(x))

In [26]:
# type(paytm_data['Transaction_Date'][0])
paytm_data.head()

,Transaction_ID,Order_ID,Payment_Mode,Issuing_Bank,POS_ID,External_Serial_No,Credit/Debit_Card_Last_4_Digits,Bank_Transaction_ID,Settled_Amount,Auth_Code,Bank/Gateway,Card_Scheme,User_Expected_Credit_Date,Standardized DateTime,UnixTimestamp,UnixTimestampAmount,DateTimeAmount,FinalAmount
0,20231201010880000937176714792993815,2023120102461608140510820434,DEBIT_CARD,HDFC,10820434,3120110745724962821,6207,81405,388.0,085174,HFPP,MASTER,DEBIT_CARD:null,20231201024543,1.701379e+09,1701378943 388.00,20231201024543 388.00,388.00
1,20231201010910000937272200426767146,ORD1538522,CREDIT_CARD,AXIS,10986499,3120112495789276165,4429,41374,600.0,127986,HFPP,VISA,CREDIT_CARD:null,20231201090508,1.701402e+09,1701401708 600.00,20231201090508 600.00,600.00
2,20231201010810000937237335752680236,ORD1538384,CREDIT_CARD,HDFC,10986499,3120111849129772033,5667,41371,150.0,029535,HFPP,MASTER,CREDIT_CARD:null,20231201064637,1.701393e+09,1701393397 150.00,20231201064637 150.00,150.00
3,20231201010930000937195176500533947,2023120103594308140610820434,CREDIT_CARD,ICICI,10820434,3120111078180630529,9002,81406,226.0,026348,HFPP,VISA,CREDIT_CARD:null,20231201035905,1.701383e+09,1701383345 226.00,20231201035905 226.00,226.00
4,20231201010820000937194197378068590,ORD1538371,CREDIT_CARD,ICICI,10986499,3120111061469625345,9002,41370,850.0,251481,HFPP,VISA,CREDIT_CARD:null,20231201035511,1.701383e+09,1701383111 850.00,20231201035511 850.00,850.00


In [27]:
# We can't use Transaction # column as there are duplicates
col_name = 'Transaction #'
print("Total entries in the ",col_name," column :",oms_data[col_name].size, " | Total unique : ", oms_data[col_name].unique().size)

# Code to find the duplicate entries 
# oms_data['Transaction #'][oms_data['Transaction #'].duplicated(keep=False)].unique()

Total entries in the  Transaction #  column : 6680  | Total unique :  6679


In [28]:
## printing potential columns which can act as unique key
def get_potential_cols_with_unique_keys(df):
    potential_cols_with_unique_keys = []
    for col in df.columns:
        if(df[col].size == df[col].unique().size):
            print("Total entries in the ",col," column :",df[col].size, " | Total unique : ", df[col].unique().size)
            potential_cols_with_unique_keys.append(col)
    return potential_cols_with_unique_keys

print("Potential unique key columns in oms_data : ",get_potential_cols_with_unique_keys(oms_data))
print("Potential unique key columns in paytm_data : ",get_potential_cols_with_unique_keys(paytm_data))


Total entries in the  Receipt #  column : 6680  | Total unique :  6680
Potential unique key columns in oms_data :  ['Receipt #']
Total entries in the  Transaction_ID  column : 4652  | Total unique :  4652
Total entries in the  Order_ID  column : 4652  | Total unique :  4652
Total entries in the  External_Serial_No  column : 4652  | Total unique :  4652
Total entries in the  UnixTimestampAmount  column : 4652  | Total unique :  4652
Total entries in the  DateTimeAmount  column : 4652  | Total unique :  4652
Potential unique key columns in paytm_data :  ['Transaction_ID', 'Order_ID', 'External_Serial_No', 'UnixTimestampAmount', 'DateTimeAmount']


### Potential Unique key columns in oms_data:
- Receipt #

### Potential Unique key columns in paytm_data:
- Transaction_ID
- Order_ID
- External_Serial_No
  

# Saving the cleaned data to /data/processed folder

In [29]:
# Saving the dataframe to .csv file for faster operations as .csv file is very lightweight format.

oms_data.to_csv("../data/processed/oms_data.csv", index=False)
paytm_data.to_csv("../data/processed/paytm_data.csv", index=False)

In [30]:
type(paytm_data['FinalAmount'][0])

str